<a href="https://colab.research.google.com/github/eunbi1/Levy_combined/blob/main/BERT_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BERT Assignment 

The task objective is to code sentiment analysis by BERT 

* An assignment part is denoted by (Assingment) 

* Grading criteria: Points are given if all your code in this notebook is runnable and the final ppl is lower than 150

* Points are not given if the testing cell at the end of the notebook is modified or extra cells (including text) are added after the last cell. Do not change N_EPOCHS for testing efficiently.

* Testing your model with the testing cell is recommended. 

* Please do not re-use the code from the example code. You have to write the code yourself.


#1. Preparing Data



In [45]:
import torch

import random
import numpy as np



In [46]:
pip install torchtext==0.10.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


 We get the pre-trained model by loading the pre-trained tiny-bert

In [48]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')


* tokenizer.tokenizer(string)
* tokenizer.convert_tokens_to_idex(tokens)
* tokenier.cls_token =[CLS]
* tokenizer.cls_token_id =101
* tokenizer.max_model_input_size = 512

The tokenizer has a vocab attribute. Using the tokenizer is as simple as calling tokenizer.tokenize on a string. This will tokenize and lower case the data in a way that is consistent with the pre-trained transformer model.

In [49]:
print('The vocab size is',len(tokenizer.vocab))
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

The vocab size is 30522
['hello', 'world', 'how', 'are', 'you', '?']


We can numericalize tokens using our vocabulary using tokenizer.convert_tokens_to_ids.

In [50]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


In [51]:
init_token = tokenizer.cls_token # cls: concatenation of the special classification token
eos_token = tokenizer.sep_token # sep: special separation token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


We can get the indexes of the special tokens by converting them using the vocabulary...

In [52]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [53]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    return tokens

We pass our `tokenize_and_cut` function as the tokenizer. The `preprocessing` argument is a function that takes in the example after it has been tokenized, this is where we will convert the tokens to their indexes. 
We define the label field as before.

In [54]:
from torchtext.legacy import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [55]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split()

In [56]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [57]:
print(vars(train_data.examples[6]))
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

{'text': [2023, 6966, 2033, 17111, 2172, 1997, 2745, 3453, 1005, 1055, 10231, 7685, 1005, 6530, 5353, 1005, 2007, 2009, 1005, 1055, 9643, 2394, 2659, 5166, 2514, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 15847, 1045, 2442, 2360, 1045, 2572, 1037, 5470, 1997, 2119, 14427, 1998, 3809, 2143, 1012, 1045, 9120, 1010, 2360, 1010, 1005, 17183, 14088, 1005, 2005, 2009, 1005, 1055, 1999, 23606, 18679, 1998, 1005, 2197, 2160, 2006, 1996, 2187, 1005, 2005, 2009, 1005, 1055, 11591, 14477, 21010, 2098, 24083, 1012, 1998, 2151, 2193, 1997, 1999, 15338, 3512, 1998, 6233, 12077, 3059, 6714, 12446, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 2023, 2001, 2074, 23100, 2295, 1012, 2785, 1997, 2066, 1037, 2329, 5166, 2472, 2245, 1005, 2292, 1005, 1055, 12661, 1000, 1045, 13183, 2006, 2115, 6542, 1000, 2302, 2437, 2009, 2205, 24560, 2075, 2085, 2008, 5469, 2003, 2067, 1999, 4827, 2007, 21071, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 2878, 2518, 2074, 2987, 1005, 1056

In [58]:
LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [59]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

#2. Build the Model


## 2.1 Load the pre-trained model

In [60]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('prajjwal1/bert-tiny')

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## (Assignment) 2.2 Make BERTGRUSentiment 

1) use the pre-trained transformer model to get embedding for the text. 

2) These embeddings will be fed into a GRU(Gated recurrent units) to produce a prediction for the sentiment of the input sentence. 

3) Within the forward pass, you should warp the transformer in a `no_grad` to ensure no gradients are calculated over this part of the model. 

* embedding dimension = hidden_size 
* use a standard initialization method 


* Output of the transfomrer is the embeddings for the whold sentence

In [61]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

##2.3 Create an instance of the model 

In [62]:
HIDDEN_DIM = 64
OUTPUT_DIM = 1
N_LAYERS = 1
BIDIRECTIONAL = True
DROPOUT = 0.1

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [63]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,460,545 trainable parameters


## 2.4 Freeze parameters 

* Set their requries_grad attribute to False 
* To do this, we simply loop through all of the named_parameters in your model. 
In order to freeze paramers (not train them) we need to set their `requires_grad` attribute to `False`. To do this, we simply loop through all of the `named_parameters` in our model.

In [64]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [65]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 74,625 trainable parameters


#3. Train the Model


In [66]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [67]:
criterion = nn.BCEWithLogitsLoss()

## (Assingment) 3.1 Check whether GPU is avaliable 

In [68]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## (Assingmnet) 3.2 Upload your model onto the device

In [69]:
model = model.to(device)
criterion = criterion.to(device)

## (Assingmnet) 3.3 Make Binary_accuracy 
* Define functions for calculating accuracy

In [70]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

## (Assignment) 3.4 Make train funtion 

In [71]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [72]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [73]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## (Assignment) 3.5 Train your model 
* This proecess takes considerably longer than any of the previous models due to the size of the transformer.

In [76]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 16s
	Train Loss: 0.555 | Train Acc: 71.64%
	 Val. Loss: 0.540 |  Val. Acc: 72.64%
Epoch: 02 | Epoch Time: 0m 16s
	Train Loss: 0.522 | Train Acc: 74.06%
	 Val. Loss: 0.521 |  Val. Acc: 73.93%
Epoch: 03 | Epoch Time: 0m 16s
	Train Loss: 0.501 | Train Acc: 75.60%
	 Val. Loss: 0.526 |  Val. Acc: 73.84%
Epoch: 04 | Epoch Time: 0m 16s
	Train Loss: 0.469 | Train Acc: 77.72%
	 Val. Loss: 0.483 |  Val. Acc: 76.66%
Epoch: 05 | Epoch Time: 0m 16s
	Train Loss: 0.441 | Train Acc: 79.54%
	 Val. Loss: 0.437 |  Val. Acc: 79.77%


In [77]:
acc= train_acc
print(train_acc)

0.7954121710610216
